In [2]:
import os
from pathlib import Path

try:
    from dotenv import find_dotenv, load_dotenv  # type: ignore
except ImportError as e:
    raise RuntimeError(
        "python-dotenv is not installed. Install it with: pip install python-dotenv"
    ) from e

# Try to load .env from the notebook's current working directory
env_path = find_dotenv(filename=".env", usecwd=True)
if env_path:
    load_dotenv(env_path)
    print(f"✅ Loaded .env: {env_path}")
else:
    # Still attempt default behavior (may load from other locations)
    load_dotenv()
    print(f"⚠️  No .env found via find_dotenv(). CWD: {Path.cwd()}")

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise RuntimeError(
        "Missing GOOGLE_API_KEY. Ensure your .env is in the current working directory "
        f"({Path.cwd()}) and contains GOOGLE_API_KEY=... then restart the kernel and re-run this cell."
    )

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
print("✅ Setup and authentication complete.")


✅ Loaded .env: c:\Kaggle-agent\.env
✅ Setup and authentication complete.


In [ ]:
from typing import Any, Dict
import json
import requests
import subprocess
import time
import uuid
import warnings

from google.adk.agents import Agent, LlmAgent
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.models.google_llm import Gemini
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from google.genai import types
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.adk.agents import ParallelAgent, SequentialAgent
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)
from google.adk.a2a.utils.agent_to_a2a import to_a2a

warnings.filterwarnings("ignore")

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [4]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [5]:
memory_service = (
    InMemoryMemoryService()
)  

In [6]:
# HELPER FUNCTION

async def run_session(runner_instance: Runner, user_queries: list[str] | str, session_id: str = "default"):
    """Helper function to run queries in a session and display responses."""
    print(f"\n### Session: {session_id}")

    # Create or retrieve session
    try:
        session = await session_service.create_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )
    except:
        session = await session_service.get_session(
            app_name=APP_NAME, user_id=USER_ID, session_id=session_id
        )

    # Convert single query to list
    if isinstance(user_queries, str):
        user_queries = [user_queries]

    # Process each query
    for query in user_queries:
        print(f"\nUser > {query}")
        query_content = types.Content(role="user", parts=[types.Part(text=query)])

        # Stream agent response
        async for event in runner_instance.run_async(
            user_id=USER_ID, session_id=session.id, new_message=query_content
        ):
            if event.is_final_response() and event.content and event.content.parts:
                text = event.content.parts[0].text
                if text and text != "None":
                    print(f"Model: > {text}")


print("✅ Helper functions defined.")

✅ Helper functions defined.


___________________                     

In [9]:
# ADD MEMORY TO AGENT

# Define constants used throughout the notebook
APP_NAME = "MemoryDemoApp"
USER_ID = "demo_user"

# Create agent
user_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="MemoryDemoAgent",
    instruction="Answer user questions in simple words.",
)

print("✅ Agent created")

✅ Agent created


In [10]:
# Create Session Service
session_service = InMemorySessionService()  # Handles conversations

# Create runner with BOTH services
runner = Runner(
    agent=user_agent,
    app_name="MemoryDemoApp",
    session_service=session_service,
    memory_service=memory_service,  # Memory service is now available!
)

print("✅ Agent and Runner created with memory support!")

✅ Agent and Runner created with memory support!


In [11]:
# User tells agent about their favorite color
await run_session(
    runner,
    "My favorite series is Stranger Things. Can you write a Haiku about it?",
    "conversation-01",  # Session ID
)


### Session: conversation-01

User > My favorite series is Stranger Things. Can you write a Haiku about it?
Model: > Upside Down's dark pull,
Friendship fights the monstrous threat,
Lights flicker, they're brave.


In [12]:
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="conversation-01"
)

# Let's see what's in the session
print("📝 Session contains:")
for event in session.events:
    text = (
        event.content.parts[0].text[:60]
        if event.content and event.content.parts
        else "(empty)"
    )
    print(f"  {event.content.role}: {text}...")

📝 Session contains:
  user: My favorite series is Stranger Things. Can you write a Haiku...
  model: Upside Down's dark pull,
Friendship fights the monstrous thr...


In [13]:
# This is the key method!
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="conversation-01"
)
await memory_service.add_session_to_memory(session)

print("✅ Session added to memory!")

✅ Session added to memory!


In [14]:
# RETRIEVING MEMORY 

# Create agent
user_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="MemoryDemoAgent",
    instruction="Answer user questions in simple words. Use load_memory tool if you need to recall past conversations.",
    tools=[
        load_memory
    ],  # Agent now has access to Memory and can search it whenever it decides to!
)

print("✅ Agent with load_memory tool created.")

✅ Agent with load_memory tool created.


In [16]:
# Create a new runner with the updated agent
runner = Runner(
    agent=user_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service,
)

await run_session(runner, "What is my favorite series?", "color-test")


### Session: color-test

User > What is my favorite series?
Model: > I see that your favorite series is Stranger Things.


In [ ]:
await run_session(runner, "My birthday is on December 15th.", "birthday-session-01")

In [ ]:
# Manually save the session to memory
birthday_session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="birthday-session-01"
)

await memory_service.add_session_to_memory(birthday_session)

print("✅ Birthday session saved to memory!")

In [ ]:
# Test retrieval in a NEW session
await run_session(
    runner, "When is my favorite series?", "birthday-session-03"  # Different session ID
)

In [17]:
# Search for series preferences
search_response = await memory_service.search_memory(
    app_name=APP_NAME, user_id=USER_ID, query="What is the user's favorite series?"
)

print("🔍 Search Results:")
print(f"  Found {len(search_response.memories)} relevant memories")
print()

for memory in search_response.memories:
    if memory.content and memory.content.parts:
        text = memory.content.parts[0].text[:80]
        print(f"  [{memory.author}]: {text}...")

🔍 Search Results:
  Found 2 relevant memories

  [user]: My favorite series is Stranger Things. Can you write a Haiku about it?...
  [MemoryDemoAgent]: Upside Down's dark pull,
Friendship fights the monstrous threat,
Lights flicker,...


____________            

In [7]:
# AUTO SAVE TO MEMORY

async def auto_save_to_memory(callback_context):
    """Automatically save session to memory after each agent turn."""
    await callback_context._invocation_context.memory_service.add_session_to_memory(
        callback_context._invocation_context.session
    )

print("✅ Callback created.")

✅ Callback created.


In [8]:
APP_NAME = "AutoMemoryApp"
USER_ID = "demo_user"

In [19]:
import litellm

litellm.set_verbose = False
litellm.suppress_debug_info = True

from google.adk.models.lite_llm import LiteLlm
from google.adk.agents import LlmAgent
import os

llm_model = LiteLlm(
    model="openrouter/meta-llama/llama-3.3-70b-instruct:free",
    # model = "openrouter/google/gemini-2.0-flash-exp:free",
    # model = "openrouter/nousresearch/hermes-3-llama-3.1-405b:free",
    # model = "openrouter/meta-llama/llama-3.2-3b-instruct:free",
    api_key=os.getenv("OPENROUTER_API_KEY"),
    api_base="https://openrouter.ai/api/v1",
)

# Agent with automatic memory saving
auto_memory_agent = LlmAgent(
    model=llm_model,
    name=APP_NAME,
    instruction="Answer user questions.",
    tools=[preload_memory],
    after_agent_callback=auto_save_to_memory,
)

print("✅ Agent created with LiteLLM and automatic memory saving!")


✅ Agent created with LiteLLM and automatic memory saving!


In [20]:
# # Agent with automatic memory saving
# auto_memory_agent = LlmAgent(
#     model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
#     name=APP_NAME,
#     instruction="Answer user questions.",
#     tools=[preload_memory],
#     after_agent_callback=auto_save_to_memory,  # saves after each turn
# )

# print("✅ Agent created with automatic memory saving!")

In [21]:
session_service = InMemorySessionService()  # Handles conversations

In [22]:
# Create a runner for the auto-save agent
# This connects our automated agent to the session and memory services
auto_runner = Runner(
    agent=auto_memory_agent,  # Use the agent with callback + preload_memory
    app_name=APP_NAME,
    session_service=session_service,  # Same services from Section 3
    memory_service=memory_service,
)

print("✅ Runner created.")

✅ Runner created.


In [23]:
# Test 1: Tell the agent about a gift (first conversation)
# The callback will automatically save this to memory when the turn completes
await run_session(
    auto_runner,
    "I got an offer from Brillar yesterday!",
    "auto-save-test",
)


### Session: auto-save-test

User > I got an offer from Brillar yesterday!
Model: > Congratulations again on the job offer from Brillar. You had mentioned this yesterday, and I'm excited to hear that it's still on your mind. How are you feeling about the offer today? Have you had a chance to think it over and consider whether you'll be accepting it, or are you still weighing your options?


In [47]:
# Test 1: Tell the agent about a gift (first conversation)
# The callback will automatically save this to memory when the turn completes
await run_session(
    auto_runner,
    "My favourite color is blue!",
    "auto-save-test",
)


### Session: auto-save-test

User > My favourite color is blue!
Model: > Blue is a wonderful color. I'll make sure to... well, not exactly "remember" it, since I don't retain information from previous conversations, but I'm happy to chat with you about it now! What do you like most about the color blue?


In [ ]:
# # User tells agent about their favorite color
# await run_session(
#     auto_runner,
#     "My favorite series is Stranger Things. Can you write a Haiku about it?",
#     "conversation-01",  # Session ID
# )


### Session: conversation-01

User > My favorite series is Stranger Things. Can you write a Haiku about it?
Model: > Here's a Haiku for Stranger Things:

Upside Down darkness
Eleven's powers shine bright
Hawkins' strange delight


In [23]:
# Test 2: Ask about the gift in a NEW session (second conversation)
# The agent should retrieve the memory using preload_memory and answer correctly
await run_session(
    auto_runner,
    "Whats my favourite color?",
    "auto-save-test-2",  # Different session ID - proves memory works across sessions!
)


### Session: auto-save-test-2

User > Whats my favourite color?
Model: > I don't have that information. As the AutoMemoryApp agent, I don't retain any information from previous conversations. Although you mentioned earlier that your favorite color is blue, I don't have the ability to recall that information since it was from a previous conversation. If you'd like to share it with me again, I'd be happy to chat with you about it!


In [13]:
# Test 2: Ask about the gift in a NEW session (second conversation)
# The agent should retrieve the memory using preload_memory and answer correctly
await run_session(
    auto_runner,
    "where do i get the offer?",
    "auto-save-test-2",  # Different session ID - proves memory works across sessions!
)


### Session: auto-save-test-2

User > where do i get the offer?
Model: > You received the job offer from Brillar. You mentioned it in our previous conversation that you got the offer from them yesterday.


___________________________________________________________________     

In [ ]:
# Tech Researcher: Focuses on AI and ML trends.
tech_researcher = Agent(
    name="TechResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research",  # The result of this agent will be stored in the session state with this key.
)

print("✅ tech_researcher created.")

✅ tech_researcher created.


In [ ]:
# Health Researcher: Focuses on medical breakthroughs.
health_researcher = Agent(
    name="HealthResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research",  # The result will be stored with this key.
)

print("✅ health_researcher created.")

✅ health_researcher created.


In [ ]:
# Finance Researcher: Focuses on fintech trends.
finance_researcher = Agent(
    name="FinanceResearcher",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research",  # The result will be stored with this key.
)

print("✅ finance_researcher created.")

✅ finance_researcher created.


In [ ]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
# Added preload_memory tool and auto_save callback for memory integration
aggregator_agent = LlmAgent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    # It uses placeholders to inject the outputs from the parallel agents, which are now in the session state.
    instruction="""Combine these three research findings into a single executive summary:

    **Technology Trends:**
    {tech_research}
    
    **Health Breakthroughs:**
    {health_research}
    
    **Finance Innovations:**
    {finance_research}
    
    Your summary should highlight common themes, surprising connections, and the most important key takeaways from all three reports. The final summary should be around 200 words.
    
    You can also use preload_memory to recall any relevant past conversations or research.""",
    tools=[preload_memory],  # Memory tool to access past conversations
    after_agent_callback=auto_save_to_memory,  # Auto-save after completion
    output_key="executive_summary",  # This will be the final output of the entire system.
)

print("✅ aggregator_agent created with memory support!")

✅ aggregator_agent created with memory support!


In [ ]:
# The ParallelAgent runs all its sub-agents simultaneously.
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")

✅ Parallel and Sequential Agents created.


In [ ]:
# Use full Runner with session and memory services (instead of InMemoryRunner)
parallel_runner = Runner(
    agent=root_agent,
    app_name=APP_NAME,
    session_service=session_service,  # Reusing from memory section
    memory_service=memory_service,    # Reusing from memory section
)

In [ ]:
# Run with the helper function that handles sessions properly
await run_session(
    parallel_runner,
    "Run the daily executive briefing on Tech, Health, and Finance",
    # "What did i get yesterday?"
    "parallel-research-session"
)


### Session: default

User > What did i get yesterday?parallel-research-session
Model: > I cannot access personal information or past interactions, so I don't know what you received yesterday.

If you are referring to a specific event or a piece of information, please provide more context, and I'll do my best to help. For instance, if you're asking about a Verizon credit for an outage, I can tell you that Verizon planned to issue a $20 credit to affected customers for an outage that occurred on Wednesday, January 15, 2026, with credits being sent via text and redeemable in the My Verizon app.
Model: > Here are three key AI/ML trends for early 2026:

1.  **Agentic AI and Autonomous Systems:** AI systems are increasingly designed to make decisions and perform complex, multi-step tasks independently, acting as digital collaborators. Major companies like Microsoft, Google, and AWS are developing frameworks for these autonomous agents. This shift promises significant productivity gains by 